In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')           # Turn off the warnings. 
%matplotlib inline

In [ ]:
# kmeans 메모리 누수를 방지하기 위한 설정.
os.environ['OMP_NUM_THREADS']='1'

In [ ]:
import matplotlib.font_manager as fm
# 한글 폰트 설정 (Windows의 경우 예시)
font_path = 'C:/Windows/Fonts/malgun.ttf'  # 'Malgun Gothic' 폰트 경로
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

In [ ]:
csv_file_path = pd.read_csv('C:/Users/yellni/Desktop/XX401/Data/인구_교통_전기차_충전소_타입_앵커시설_fin.csv')                                 
#df.head()
csv_file_path.head()

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# CSV 파일 경로 설정
csv_file_path = 'C:/Users/yellni/Desktop/XX401/Data/인구_교통_전기차_충전소_타입_앵커시설_fin.csv'

# 필요한 열만 선택하여 데이터프레임으로 읽기
columns_to_select = ["총 인구수", "총출근발생량", "전기차 수","완속","앵커시설", "충전소 개수"]
df = pd.read_csv(csv_file_path)
x=df[columns_to_select]

# KMeans 클러스터링
kmeans = KMeans(n_clusters=4)  
df['클러스터'] = kmeans.fit_predict(x)

# 클러스터링 결과 출력
print(x)

# 클러스터링 결과 시각화
plt.scatter(df['총 인구수'], df['총출근발생량'],c=df['클러스터'], cmap='RdYlBu')  #Spectral: 여러색 변경
plt.title('K-Means Clustering')
plt.show()

In [ ]:
# 군집의 수를 키워가며 Sum of Squares Within (~ Internal Sum of Squares)을 계산한다.
ssw = []
cluster_ns = range(2,16)
for n in cluster_ns:
    my_cluster = KMeans(n)
    my_cluster.fit(x)
    ssw.append(my_cluster.inertia_)         # SSW를 모아둔다.

plt.figure(figsize = (6,6))
plt.plot(cluster_ns, ssw)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('Sum of Squares Within')
plt.show()

In [ ]:
from sklearn.decomposition import PCA  
# 주성분 분석 (PCA)를 활용한 차원축소 (2차원).
my_pca = PCA(n_components = 2)   
transformed_comps = my_pca.fit_transform(x)         # Transformed 된 좌표.
df_transformed_comps = pd.DataFrame(data = transformed_comps, columns = ['PC1', 'PC2'])  # PC = Principal Component = 주성분.
#df_transformed_comps.head(10)
df_transformed_comps=df_transformed_comps.join(pd.Series(df['클러스터'], name='cluster_label'))
df_transformed_comps.head(10)

In [ ]:
# 산점도 시각화.
my_colors = {0:'red',1:'blue',2:'purple',3:'green'}    # 군집 label => Color.
#my_names = {0: 'Moderate unknwons.', 1: 'Social minimalists.', 2: 'Overseas.', 3: 'Hyper-mobile and hyper-social.'} # 군집 label => 명칭.

plt.figure(figsize = (6,6))
for a_cluster_n, df_small in df_transformed_comps.groupby('cluster_label'):
#    plt.scatter('PC1', 'PC2', data = df_small, c = my_colors[a_cluster_n], label = my_names[a_cluster_n], s = 10, alpha=0.6 )
    plt.scatter(df_small['PC1'], df_small['PC2'], c = my_colors[a_cluster_n], s=10, alpha=0.6, marker="s" ) # label = my_names[a_cluster_n]

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('K-means')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import json
import folium

# 경기도 지도 만들기
g_map = folium.Map(location=[37.5502, 126.982], zoom_start=9)
g_map

In [ ]:
pip install geopandas folium

In [ ]:
import json
import folium

with open('C:/Users/yellni/Desktop/XX401/Data/HangJeongDong_ver20230701.geojson',encoding='utf-8') as response:
    geo_json = json.load(response)

# 경기도 시군구 경계 정보를 가진 geo-json 파일 불러오기
# 경기도 행정구역 경계 지리 정보를 사용

geo_json

In [ ]:
g_map = folium.Map(location=[37.5502, 126.982], zoom_start=9)
g_map

In [ ]:
folium.Choropleth(
    geo_data=geo_json,
    data=clustered_df,
    columns=['시군구', '클러스터'],
    key_on="properties.sggnm",
    nan_fill_color='white',
    nan_fill_opacity=0.0,
    fill_opacity=0.7,  # 색 투명도
    line_opacity=0.0,  # 선 투명도
    legend_name='클러스터',  # 범례
    fill_color='Set2'
).add_to(g_map)

# 지도 출력
g_map.save('C:/Users/yellni/Desktop/XX401/Data/경기도_클러스터링_결과_Set2.html')
g_map